Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

#Model Building

In [3]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.6

In [5]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /databricks/driver/aml_config/config.json
Workspace name: AMLSworkspace
Azure region: westeurope
Resource group: resgrpAMLS

In [6]:
#get the train and test datasets
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (25967, 15)
test: (6594, 15)
root
-- marital_status: string (nullable = true)
-- education_num: integer (nullable = true)
-- capital_gain: integer (nullable = true)
-- education: string (nullable = true)
-- workclass: string (nullable = true)
-- age: integer (nullable = true)
-- capital_loss: integer (nullable = true)
-- occupation: string (nullable = true)
-- sex: string (nullable = true)
-- native_country: string (nullable = true)
-- race: string (nullable = true)
-- hours_per_week: integer (nullable = true)
-- fnlwgt: integer (nullable = true)
-- relationship: string (nullable = true)
-- income: string (nullable = true)

#Define Model

In [8]:
label = "income"
dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip"))
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [9]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil

model_name = "AdultCensus_runHistory.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "Ignite_AI_Talk")
root_run = myexperiment.start_logging()

# Regularization Rates - 
regs = [0.0001, 0.001, 0.01, 0.1]

# try a bunch of regularization rate in a Logistic Regression model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = LogisticRegression(regParam=reg)
        
        # put together the pipeline
        pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

        # train the model
        model_p = pipe.fit(train)
        
        # make prediction
        pred = model_p.transform(test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", train.columns)

        # save model
        model_p.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        #run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0001
Area under ROC: 0.9064101633196264
Area Under PR: 0.7726472151837128
Regularization rate: 0.001
Area under ROC: 0.9058318712410425
Area Under PR: 0.7703864183790115
Regularization rate: 0.01
Area under ROC: 0.9024334945586245
Area Under PR: 0.7594976345960406
Regularization rate: 0.1
Area under ROC: 0.8938250557175418
Area Under PR: 0.735319095123455
run id: 5bbecaee-8225-4f18-81f3-0e389c55359e

In [10]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [11]:
metrics = root_run.get_metrics(recursive=True)
best_run_id = max(metrics, key = lambda k: metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', metrics[best_run_id]['au_roc'], metrics[best_run_id]['reg'])

Best run is: 179026e2-c226-4386-b71a-6330f9000786
Metrics: 0.9064101633196264 0.0001

In [12]:
#Download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

In [14]:
##unzip the model to dbfs (as load() seems to require that) and load it.
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_p_best = PipelineModel.load(model_name)

In [15]:
# make prediction
pred = model_p_best.transform(test)
output = pred[['hours_per_week','age','marital_status','income','prediction']]
display(output.limit(8))

hours_per_week,age,marital_status,income,prediction
40,64,Divorced,<=50K,0.0
30,55,Divorced,<=50K,0.0
35,45,Divorced,<=50K,0.0
40,58,Divorced,<=50K,0.0
40,57,Divorced,<=50K,0.0
30,38,Divorced,<=50K,0.0
40,27,Divorced,<=50K,0.0
40,42,Divorced,<=50K,0.0


In [16]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.9064101633196264
Area Under PR: 0.7726472151837128

#Model Persistence

In [18]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_p_best.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/AdultCensus_runHistory.mml

In [19]:
%sh

ls -la /dbfs/AdultCensus_runHistory.mml/*

/dbfs/AdultCensus_runHistory.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Jan 8 09:14 .
drwxr-xr-x 1 root root 0 Jan 8 09:14 ..
-rw-r--r-- 1 root root 736 Jan 8 09:14 part-00000
-rw-r--r-- 1 root root 0 Jan 8 09:14 _SUCCESS

/dbfs/AdultCensus_runHistory.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Jan 8 09:14 .
drwxr-xr-x 1 root root 0 Jan 8 09:14 ..
drwxr-xr-x 1 root root 0 Jan 8 09:14 00_StringIndexer_d94a21041c04
drwxr-xr-x 1 root root 0 Jan 8 09:14 01_StringIndexer_61958fd0a7a8
drwxr-xr-x 1 root root 0 Jan 8 09:14 02_StringIndexer_3c905f5ca5d1
drwxr-xr-x 1 root root 0 Jan 8 09:14 03_StringIndexer_9dda09a3cc8f
drwxr-xr-x 1 root root 0 Jan 8 09:14 04_StringIndexer_f119fbfddf1d
drwxr-xr-x 1 root root 0 Jan 8 09:14 05_StringIndexer_6069c53520c8
drwxr-xr-x 1 root root 0 Jan 8 09:14 06_StringIndexer_2e74724c1b10
drwxr-xr-x 1 root root 0 Jan 8 09:14 07_StringIndexer_4bfd943e9aa8
drwxr-xr-x 1 root root 0 Jan 8 09:14 08_OneHotEncoder_3aa16ee35398
drwxr-xr-x 1 root root 0 Jan 8 09:14 09_OneHotEncoder_e232572c2600
drwxr-xr-x 1 root root 0 Jan 8 09:14 10_OneHotEncoder_1f5f2bf626b6
drwxr-xr-x 1 root root 0 Jan 8 09:14 11_OneHotEncoder_cac0850fe818
drwxr-xr-x 1 root root 0 Jan 8 09:14 12_OneHotEncoder_89c0b0ce568d
drwxr-xr-x 1 root root 0 Jan 8 09:14 13_OneHotEncoder_17b6f04a68db
drwxr-xr-x 1 root root 0 Jan 8 09:14 14_OneHotEncoder_e399fc2dc611
drwxr-xr-x 1 root root 0 Jan 8 09:14 15_OneHotEncoder_1503f819eed0
drwxr-xr-x 1 root root 0 Jan 8 09:14 16_StringIndexer_11bf439d6c02
drwxr-xr-x 1 root root 0 Jan 8 09:14 17_VectorAssembler_2dc9d60bd405
drwxr-xr-x 1 root root 0 Jan 8 09:14 18_LogisticRegression_6383d3e26640

In [20]:
dbutils.notebook.exit("success")

success